In [9]:
p1 = 0.1
p1

0.1

In [21]:
g = lambda p1: 2 * p1 * (1-p1)
g(p1)

0.18000000000000002

In [35]:
import pandas as pd
from sklearn import *

In [18]:
df = pd.read_csv("/data/credit-default.csv")
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [19]:
target = "default"
X = df.drop(columns=[target]) # Features
y = preprocessing.LabelEncoder().fit_transform(df[target]) # target variable

X = pd.get_dummies(X, drop_first=True) # One hot encoding

# Randomly split the data into training and test test. 
# Keeping 30% of the records in test set.
# random_state creates a reproducible set of random samples 
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y
                                , test_size = 0.3, random_state = 1)

In [20]:
pd.Series(y_train).value_counts()

0    486
1    214
dtype: int64

In [22]:
p1 = 214 / 700
g(p1)

0.4245061224489796

In [24]:
g1 = g(179/424)
g1

0.48788492346030615

In [25]:
g2 = g(35/276)
g2

0.2214608275572359

In [26]:
info_gain1 = 0.4245061224489796 - (g1 + g2)/2
info_gain1

0.0698332469402086

In [28]:
bucket1 = y_train[X_train.months_loan_duration<22.5]
pd.Series(bucket1).value_counts()

0    310
1     97
dtype: int64

In [29]:
g1 = g(97/(97+310))
g1

0.3630568249732869

In [30]:
bucket2 = y_train[X_train.months_loan_duration>=22.5]
pd.Series(bucket2).value_counts()

0    176
1    117
dtype: int64

In [31]:
g2 = g(117/(117+176))
g2

0.47972603058859165

In [33]:
info_gain2 = 0.4245061224489796 - (g1+g2)/2
info_gain2

0.0031146946680403187

In [37]:
import numpy as np

In [54]:
pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    #("scaler", preprocessing.StandardScaler()),
    ("est", tree.DecisionTreeClassifier(max_depth=3, random_state=1))
])
pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

y_test_prob = pipe.predict_proba(X_test)[:, 1]
y_test_pred = np.where(y_test_prob > 0.5, 1, 0)
print(metrics.confusion_matrix(y_test, y_test_pred))
print("Accuracy:", metrics.accuracy_score(y_test, y_test_pred))
print("Precision:", metrics.precision_score(y_test, y_test_pred))
print("Recall:", metrics.recall_score(y_test, y_test_pred))
print("F1:", metrics.f1_score(y_test, y_test_pred))

[[191  23]
 [ 57  29]]
Accuracy: 0.7333333333333333
Precision: 0.5576923076923077
Recall: 0.3372093023255814
F1: 0.4202898550724638


In [55]:
est = pipe.steps[-1][-1]
est

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [56]:
summary = pd.DataFrame({"feature": X.columns, "importance": est.feature_importances_})
summary = summary.sort_values("importance", ascending = False)
summary

,feature,importance
9,checking_balance_unknown,0.526610
0,months_loan_duration,0.181563
24,savings_balance_< 100 DM,0.085239
11,credit_history_fully repaid,0.084542
4,age,0.062193
39,installment_plan_none,0.059853
36,property_other,0.000000
28,employment_length_4 - 7 yrs,0.000000
29,employment_length_> 7 yrs,0.000000
30,employment_length_unemployed,0.000000


In [57]:
from sklearn.tree import export_graphviz
export_graphviz(est, out_file = "tree.dot", feature_names = X.columns, filled=True)
!dot -Tpng tree.dot -o tree.png

In [53]:
!ls -lh

total 3576
-rw-r--r--@ 1 abasar  staff   9.3K Oct 22 17:24 Day 1 - Spark.ipynb
-rw-r--r--  1 abasar  staff   170K Oct 22 16:26 Day 1 - handle null.ipynb
-rw-r--r--  1 abasar  staff   143K Oct 22 11:36 Day 1 - pandas.ipynb
-rw-r--r--@ 1 abasar  staff    59K Oct 22 13:48 Day 1 - top 10 movies.ipynb
-rw-r--r--  1 abasar  staff   247K Oct 22 15:04 Day 1 - visualization.ipynb
-rw-r--r--  1 abasar  staff    47K Oct 23 15:34 Day 2 - regression.ipynb
-rw-r--r--  1 abasar  staff    63K Oct 24 12:32 Day 3 - Credit fraud detection.ipynb
-rw-r--r--  1 abasar  staff    70K Oct 24 12:54 Day 3 - classification problem (credit default).ipynb
-rw-r--r--  1 abasar  staff    28K Oct 24 14:53 Untitled.ipynb
-rw-r--r--  1 abasar  staff   110K Oct 23 17:17 day 2 - House price prediction problem.ipynb
-rw-r--r--  1 abasar  staff    31K Oct 23 16:51 model.pkl
-rw-r--r--  1 abasar  staff    20K Oct 22 14:23 plt.jpg
-rw-r--r--@ 1 abasar  staff    12K Oct 22 14:31 plt.pdf
-rw-r--r--  1 abasar  staff   8.4K Oct 2

In [58]:
pipe = pipeline.Pipeline([
    #("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    #("scaler", preprocessing.StandardScaler()),
    ("est", ensemble.RandomForestClassifier(n_estimators=20, max_depth=6, random_state=1))
])
pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
y_test_pred = pipe.predict(X_test)

y_test_prob = pipe.predict_proba(X_test)[:, 1]
y_test_pred = np.where(y_test_prob > 0.5, 1, 0)
print(metrics.confusion_matrix(y_test, y_test_pred))
print("Accuracy:", metrics.accuracy_score(y_test, y_test_pred))
print("Precision:", metrics.precision_score(y_test, y_test_pred))
print("Recall:", metrics.recall_score(y_test, y_test_pred))
print("F1:", metrics.f1_score(y_test, y_test_pred))

[[207   7]
 [ 72  14]]
Accuracy: 0.7366666666666667
Precision: 0.6666666666666666
Recall: 0.16279069767441862
F1: 0.26168224299065423
